In [1]:
# Dependencies
import tweepy
import json
import numpy as np
import pandas as pd
from config import consumer_key, consumer_secret, access_token, access_token_secret

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, parser=tweepy.parsers.JSONParser())

#Filters
min_tweets = 2
max_tweets = 100000
lang = "en"

In [49]:
#Set Dictionary
Result = {}

#Import City Data
city_df = pd.read_csv("Capitals.csv")
city_df = city_df.set_index("Country")
#Checkpoint
n = input("Which Checkpoint?  [1~6]")

#Checkpoints for Timeframe
start_dates =  ['2018-06-14',
                '2018-06-17',
                '2018-06-20',
                '2018-06-23',
                '2018-06-27',
                '2018-06-30']

end_dates =    ['2018-06-16',
                '2018-06-19',
                '2018-06-22',
                '2018-06-26',
                '2018-06-29',
                '2018-07-03']

#Iterate through rows
for index, row in city_df.iterrows():
    tweet_list = []
    
    #Set Coordinate & Search Term
    lat = str(row["Latitude"]) + "0000000"
    lng = str(row["Longitude"]) + "0000000"
    search_term =  "World Cup geocode:"+lat+","+lng+",15km"
    
    #Pull Data
    oldest_tweet = None
    public_tweets = api.search(search_term, 
                               count=200, 
                               result_type="recent")
    
    #Loop Through the data
    for tweet in public_tweets["statuses"]:
        
        # Use filters to check if user meets conditions
        if (tweet["user"]["statuses_count"] > min_tweets and
            tweet["user"]["statuses_count"] < max_tweets and
            tweet["user"]["lang"] == lang):
            tweet_list.append(tweet["text"])
            
    Result[index] = tweet_list

Which Checkpoint?  [1~6]2


In [50]:
len(Result.keys())

32

In [51]:
Result

{'Argentina': ['RT @MayaPark28: Yes ARMY!!! We won on Instagram!!! Fake Love will be played on the World Cup!!\n\n@BTS_twt #BTSxFifaWorldCup',
  'RT @golazoargentino: #OnThisDay in 1997 José Pékerman led @Argentina under-20s to the World Cup - Riquelme, Aimar, Cambiasso, Placente, Sam…',
  'RT @MayaPark28: Yes ARMY!!! We won on Instagram!!! Fake Love will be played on the World Cup!!\n\n@BTS_twt #BTSxFifaWorldCup',
  'RT @sofyoongi: Which song should be played at the FIFA World Cup finals?\n\n  0%        BTS — Fake Love\n\n  0%        EXO — Power\n\n  0%…',
  'RT @golazoargentino: #OnThisDay in 1997 José Pékerman led @Argentina under-20s to the World Cup - Riquelme, Aimar, Cambiasso, Placente, Sam…',
  'RT @sofyoongi: Which song should be played at the FIFA World Cup finals?\n\n  0%        BTS — Fake Love\n\n  0%        EXO — Power\n\n  0%…',
  'RT @golazoargentino: #OnThisDay in 1997 José Pékerman led @Argentina under-20s to the World Cup - Riquelme, Aimar, Cambiasso, Placente, Sam…',

In [46]:
np.save(f'Result_{n}.npy', Result) 

In [47]:
read_dictionary = np.load('Result_1.npy').item()